In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP

Mounted at /content/drive
/content/drive/MyDrive/NLP


In [ ]:
!pip install kmeans_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from kmeans_pytorch import kmeans
import pandas as pd

In [ ]:
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import v_measure_score

In [ ]:
embeddings = torch.load("emb.pt", map_location = torch.device("cpu"))

In [ ]:
sentences = pd.read_csv("sentences.csv")
data = sentences[0:65536]

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=5ea66c6764f062f97780b3bdc9c5c2c64fe6bcdf9a6704edc724282fd75b1f1e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')
top_n = 3

In [ ]:
keys = torch.load("keys.pt")

for i in tqdm(range(len(data))):
    try:
      count = CountVectorizer(ngram_range=(3, 3),stop_words=None).fit([data.text[i]])
      candidates = count.get_feature_names_out()
    except ValueError as e:
      candidates = [""]
    candidate_embeddings = model.encode(candidates)
    distances=cosine_similarity([embeddings.cpu().numpy()[i]], candidate_embeddings)
    keywords_embeddings = np.array([candidate_embeddings[index] for index in distances.argsort()[0][-top_n:]])
    mean_keywords=np.mean(keywords_embeddings,axis=0)
    keys[i] = torch.tensor(mean_keywords)

torch.save(keys, "keys.pt")

100%|██████████| 27311/27311 [26:02<00:00, 17.48it/s]


In [ ]:
keys = torch.load("keys.pt")

In [ ]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(embeddings, keys)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
def q_phi(z_dim=10, x_dim=384):
  model = torch.nn.Sequential(                           
    nn.Linear(x_dim, 192),
    nn.ReLU(),
    nn.Linear(192, 96),
    nn.ReLU(),
    nn.Linear(96, 48),
    nn.ReLU(),
    nn.Linear(48, z_dim)
  )
  return model

In [ ]:
def p_theta(z_dim=10, x_dim=384):
  model = torch.nn.Sequential(
    
    nn.Linear(z_dim, 48),
    nn.ReLU(),
    nn.Linear(48, 96),
    nn.ReLU(),
    nn.Linear(96, 192),
    nn.ReLU(),
    nn.Linear(192, x_dim)
  )
  return model        

In [ ]:
def vae_loss(x, x_logit, z_mu, z_logvar, beta, predict, anti_predict, label):
  recon_loss = nn.functional.cross_entropy(x, x_logit)
  kl_loss = 0.5 * torch.mean(torch.square(z_mu) + torch.exp(z_logvar) - z_logvar - 1, axis = 1)
  prediction_loss = torch.mean(nn.functional.cosine_similarity(predict, label))
  anti_prediction_loss = torch.mean(nn.functional.cosine_similarity(anti_predict, label))
  
  vae_loss = recon_loss + torch.mean(beta * kl_loss) + prediction_loss - anti_prediction_loss
  return vae_loss

In [ ]:
class Model(nn.Module):
  def __init__(self, Encoder_Mu, Encoder_Var, Decoder, Predictor, AntiPredictor, z_dim):
        super(Model, self).__init__()
        self.Encoder_Mu = Encoder_Mu
        self.Encoder_Var = Encoder_Var
        self.Decoder = Decoder
        self.Predictor = Predictor
        self.AntiPredictor = AntiPredictor
        self.z_dim = z_dim
  
  def sample_z(self, mu, log_var):
    eps = torch.randn(mu.size())
    samples = mu + torch.exp(0.5 * log_var) * eps
    return samples
  
  def forward(self, x):
      mu = self.Encoder_Mu(x)
      var = self.Encoder_Var(x)
      z = self.sample_z(mu, var)
      x_hat = self.Decoder(z)
      predict = self.Predictor(torch.transpose(torch.transpose(z, 0, 1)[:self.z_dim-2], 0, 1))
      anti_predict = self.AntiPredictor(z)
        
      return x_hat, mu, var, predict, anti_predict

In [ ]:
enc_mu = q_phi(z_dim = 10)
enc_var = q_phi(z_dim = 10)
dec = p_theta(z_dim = 10)
predictor = p_theta(z_dim = 8)
anti_predictor = p_theta(z_dim = 10)
model = Model(enc_mu, enc_var, dec, predictor, anti_predictor, 10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 5
beta = 10

In [ ]:
for epoch in tqdm(range(epochs)):
    overall_loss = 0
    for batch_idx, (x, label) in tqdm(enumerate(train_dataloader)):
        x = x.view(64, 384)
        #x = x.to(device)

        optimizer.zero_grad()

        x_hat, mean, log_var, predict, anti_predict = model(x)
        loss = vae_loss(x, x_hat, mean, log_var, beta, predict, anti_predict, label)
      
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*64))

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
4it [00:00, 34.62it/s]
9it [00:00, 40.00it/s]
14it [00:00, 43.23it/s]
21it [00:00, 52.91it/s]
27it [00:00, 54.77it/s]
33it [00:00, 55.15it/s]
39it [00:00, 56.01it/s]
45it [00:00, 56.58it/s]
52it [00:00, 58.27it/s]
59it [00:01, 60.06it/s]
66it [00:01, 60.21it/s]
73it [00:01, 58.90it/s]
79it [00:01, 57.84it/s]
85it [00:01, 57.95it/s]
92it [00:01, 59.36it/s]
98it [00:01, 58.97it/s]
104it [00:01, 57.66it/s]
110it [00:01, 56.89it/s]
116it [00:02, 56.40it/s]
122it [00:02, 57.21it/s]
129it [00:02, 58.36it/s]
136it [00:02, 59.22it/s]
143it [00:02, 59.75it/s]
149it [00:02, 58.49it/s]
155it [00:02, 58.79it/s]
162it [00:02, 58.98it/s]
168it [00:02, 58.84it/s]
175it [00:03, 59.41it/s]
181it [00:03, 58.45it/s]
187it [00:03, 58.17it/s]
193it [00:03, 57.67it/s]
199it [00:03, 57.38it/s]
205it [00:03, 56.52it/s]
211it [00:03, 52.22it/s]
217it [00:03, 51.22it/s]
223it [00:03, 51.57it/s]
229it [00:04, 51.72it/s]
235it [00:04, 52.03it/s]
241it [00:0

	Epoch 1 complete! 	Average Loss:  -2.5515472775189178e+17



0it [00:00, ?it/s]
7it [00:00, 60.99it/s]
14it [00:00, 57.60it/s]
20it [00:00, 57.37it/s]
26it [00:00, 57.20it/s]
32it [00:00, 56.87it/s]
38it [00:00, 56.82it/s]
44it [00:00, 57.03it/s]
50it [00:00, 55.11it/s]
56it [00:00, 55.43it/s]
62it [00:01, 55.53it/s]
68it [00:01, 55.03it/s]
74it [00:01, 56.34it/s]
80it [00:01, 56.93it/s]
86it [00:01, 57.15it/s]
92it [00:01, 56.79it/s]
98it [00:01, 54.87it/s]
104it [00:01, 55.22it/s]
110it [00:01, 54.68it/s]
116it [00:02, 53.33it/s]
122it [00:02, 51.60it/s]
128it [00:02, 51.87it/s]
134it [00:02, 52.08it/s]
140it [00:02, 51.03it/s]
146it [00:02, 49.86it/s]
152it [00:02, 49.56it/s]
157it [00:02, 49.64it/s]
162it [00:03, 48.74it/s]
167it [00:03, 48.30it/s]
172it [00:03, 46.21it/s]
177it [00:03, 47.22it/s]
184it [00:03, 53.40it/s]
192it [00:03, 59.92it/s]
200it [00:03, 63.90it/s]
208it [00:03, 66.84it/s]
216it [00:03, 68.79it/s]
223it [00:03, 67.28it/s]
231it [00:04, 67.76it/s]
238it [00:04, 67.77it/s]
246it [00:04, 69.66it/s]
254it [00:04, 71.33it/

	Epoch 2 complete! 	Average Loss:  -1.199394355274972e+20



0it [00:00, ?it/s]
6it [00:00, 56.34it/s]
12it [00:00, 56.79it/s]
18it [00:00, 53.78it/s]
24it [00:00, 51.92it/s]
30it [00:00, 52.53it/s]
36it [00:00, 52.42it/s]
42it [00:00, 53.35it/s]
48it [00:00, 52.20it/s]
54it [00:01, 52.58it/s]
60it [00:01, 52.78it/s]
66it [00:01, 53.23it/s]
72it [00:01, 52.96it/s]
78it [00:01, 51.80it/s]
84it [00:01, 27.54it/s]
89it [00:02, 21.35it/s]
93it [00:02, 21.91it/s]
96it [00:02, 22.23it/s]
103it [00:02, 30.02it/s]
111it [00:02, 38.92it/s]
119it [00:02, 46.67it/s]
127it [00:03, 53.15it/s]
134it [00:03, 56.68it/s]
142it [00:03, 60.81it/s]
150it [00:03, 63.80it/s]
157it [00:03, 64.96it/s]
164it [00:03, 65.25it/s]
172it [00:03, 68.15it/s]
180it [00:03, 69.36it/s]
188it [00:03, 69.90it/s]
196it [00:04, 71.49it/s]
204it [00:04, 70.60it/s]
212it [00:04, 69.55it/s]
219it [00:04, 69.39it/s]
226it [00:04, 69.43it/s]
233it [00:04, 69.15it/s]
240it [00:04, 67.86it/s]
247it [00:04, 67.84it/s]
254it [00:04, 67.32it/s]
262it [00:04, 69.97it/s]
270it [00:05, 70.00it/s

	Epoch 3 complete! 	Average Loss:  -2.2445882715596556e+21



0it [00:00, ?it/s]
7it [00:00, 64.84it/s]
14it [00:00, 66.75it/s]
21it [00:00, 67.16it/s]
29it [00:00, 69.17it/s]
37it [00:00, 71.13it/s]
45it [00:00, 70.58it/s]
53it [00:00, 71.83it/s]
61it [00:00, 72.60it/s]
69it [00:00, 71.21it/s]
77it [00:01, 70.10it/s]
85it [00:01, 70.92it/s]
93it [00:01, 71.61it/s]
101it [00:01, 71.32it/s]
109it [00:01, 72.75it/s]
117it [00:01, 71.46it/s]
125it [00:01, 71.55it/s]
133it [00:01, 72.20it/s]
141it [00:01, 70.96it/s]
149it [00:02, 69.00it/s]
156it [00:02, 67.84it/s]
163it [00:02, 67.47it/s]
170it [00:02, 67.73it/s]
178it [00:02, 68.73it/s]
185it [00:02, 68.56it/s]
193it [00:02, 69.54it/s]
201it [00:02, 69.97it/s]
208it [00:02, 68.49it/s]
216it [00:03, 70.98it/s]
224it [00:03, 71.54it/s]
232it [00:03, 70.81it/s]
240it [00:03, 69.24it/s]
248it [00:03, 70.27it/s]
256it [00:03, 71.45it/s]
264it [00:03, 71.53it/s]
272it [00:03, 70.84it/s]
280it [00:03, 70.68it/s]
288it [00:04, 70.48it/s]
296it [00:04, 69.64it/s]
303it [00:04, 69.59it/s]
311it [00:04, 70.1

	Epoch 4 complete! 	Average Loss:  -8.4863371179566e+21



0it [00:00, ?it/s]
7it [00:00, 66.49it/s]
14it [00:00, 67.14it/s]
21it [00:00, 67.60it/s]
29it [00:00, 70.33it/s]
37it [00:00, 67.42it/s]
44it [00:00, 66.80it/s]
51it [00:00, 67.07it/s]
59it [00:00, 68.57it/s]
66it [00:00, 68.90it/s]
73it [00:01, 68.79it/s]
80it [00:01, 67.84it/s]
87it [00:01, 67.76it/s]
94it [00:01, 67.79it/s]
101it [00:01, 68.12it/s]
108it [00:01, 68.14it/s]
115it [00:01, 66.71it/s]
122it [00:01, 65.13it/s]
129it [00:01, 64.14it/s]
137it [00:02, 66.60it/s]
145it [00:02, 68.85it/s]
153it [00:02, 70.10it/s]
161it [00:02, 68.32it/s]
169it [00:02, 69.58it/s]
176it [00:02, 69.12it/s]
183it [00:02, 69.15it/s]
190it [00:02, 68.91it/s]
198it [00:02, 69.44it/s]
206it [00:03, 70.48it/s]
214it [00:03, 71.69it/s]
222it [00:03, 72.97it/s]
230it [00:03, 71.17it/s]
238it [00:03, 71.53it/s]
246it [00:03, 71.36it/s]
254it [00:03, 69.77it/s]
261it [00:03, 69.72it/s]
268it [00:03, 69.33it/s]
275it [00:04, 69.16it/s]
282it [00:04, 67.98it/s]
289it [00:04, 67.09it/s]
297it [00:04, 68.58

	Epoch 5 complete! 	Average Loss:  -1.8520119277702097e+22


In [ ]:
zs = torch.zeros(1024, 64, 10)

with torch.no_grad():
  for i, (x, label) in enumerate(train_dataloader):
    z_mu = model.Encoder_Mu(x)
    z_var = model.Encoder_Var(x)
    z_sample = model.sample_z(z_mu, z_var)
    zs[i] = z_sample

In [ ]:
torch.save(zs, "zs.pt")

In [ ]:
zs = torch.load("zs.pt")

In [ ]:
zs = zs.reshape(65536, 10)
##zs = torch.transpose(zs, 0, 1)[:8]
##zs = torch.transpose(zs, 0, 1)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters = 10)
kmeans.fit(zs)
cluster_ids_x = torch.tensor(kmeans.labels_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
data["cluster"] = cluster_ids_x.numpy()
clustered = data.groupby("post_id")['cluster'].agg(lambda x: pd.Series.mode(x)[0])

<ipython-input-96-204dd8f785cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["cluster"] = cluster_ids_x.numpy()


In [ ]:
apost = pd.read_csv("author_post.csv")
aaa = apost[0:6595]
aaa["cluster"] = clustered

<ipython-input-97-39e558b93e1e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aaa["cluster"] = clustered


In [ ]:
v_measure_score(aaa["author_id"], aaa["cluster"])

0.03041823057273694